In [1]:
%load_ext jupyter_black

python standard library

- os
- sys
- pathlib

- dataclasses

In [2]:
import os
import sys
import shutil

import gzip
import zipfile
from pathlib import Path

text = f"""
os.path = {os.path.abspath(".")}
pathlib.Path.cwd() = {Path.cwd()}
sys.path = {sys.path}
"""
print(text)
dict(line.split("=", maxsplit=1) for line in text.split("\n") if line != "")


os.path = /home/leaver2000/afit/2022-09
pathlib.Path.cwd() = /home/leaver2000/afit/2022-09
sys.path = ['/home/leaver2000/afit/2022-09', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/opt/venv/lib/python3.10/site-packages', '/opt/venv/lib/python3.10/site-packages/IPython/extensions', '/home/leaver2000/.ipython']



{'os.path ': ' /home/leaver2000/afit/2022-09',
 'pathlib.Path.cwd() ': ' /home/leaver2000/afit/2022-09',
 'sys.path ': " ['/home/leaver2000/afit/2022-09', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/opt/venv/lib/python3.10/site-packages', '/opt/venv/lib/python3.10/site-packages/IPython/extensions', '/home/leaver2000/.ipython']"}

In [3]:
cwd = Path.cwd()
assert cwd.is_dir() and not cwd.is_file()
# glob (/ɡlɑːb/) patterns specify sets of filenames with wildcard characters
# so in our working directory match all of the files or navigate to the parent and look for files ending with .py
# the glob method yields a generator function
tuple(cwd.glob("*")), tuple(cwd.parent.glob("*.py"))

((PosixPath('/home/leaver2000/afit/2022-09/lesson-2.ipynb'),
  PosixPath('/home/leaver2000/afit/2022-09/lesson-3.ipynb'),
  PosixPath('/home/leaver2000/afit/2022-09/lesson-1.ipynb')),
 (PosixPath('/home/leaver2000/afit/app.py'),))

In [4]:
import json

for file in Path.cwd().glob("*"):
    # the path object
    assert file.is_file()
    if ".ipynb" in file.suffixes:
        print(f"found a notebook {file}")
        # using the with key word will open the file as a context manager
        with file.open("r") as f:
            lines = f.readlines()
            print(f"it had {len(lines)} lines")
    # if not using the with key word
    # need to remember to close the file
    # otherwise we are loading tons of
    # unused data into memory
    f = file.open()
    f.close()

found a notebook /home/leaver2000/afit/2022-09/lesson-2.ipynb
it had 349 lines
found a notebook /home/leaver2000/afit/2022-09/lesson-3.ipynb
it had 0 lines
found a notebook /home/leaver2000/afit/2022-09/lesson-1.ipynb
it had 993 lines


In [5]:
from dataclasses import dataclass
from typing import overload

import pandas as pd
import numpy as np
from numpy.typing import NDArray

In [6]:
@dataclass(frozen=True)
class Wind:
    """
    surface wind representation
    """

    direction: float
    speed: float
    gust: float = 0.0

    def to_dict(self) -> dict[str, float]:
        return self.__dict__

    def to_numpy(self) -> np.ndarray:
        return np.asanyarray(tuple(self), dtype=np.float32)

    def to_pandas(self) -> pd.Series:
        return pd.Series(self.__dict__, dtype=np.float32)

    def __gt__(self, other: "Wind") -> bool:
        return max(self.speed, self.gust) > max(other.speed, other.gust)

    def __sub__(self, other: "Wind") -> "Wind":
        wdir, wspd, wgst = self.to_numpy() - other.to_numpy()
        wdir = (wdir + 180) % 360 - 180
        return Wind(wdir, wspd, wgst)

    def __abs__(self) -> "Wind":
        return Wind(*np.abs(tuple(self)))

    def __str__(self) -> str:
        wdir, wspd, wgst = map(int, tuple(self))
        return f"{wdir:03d}{wspd:02d}{f'G{wgst:02d}' if wgst else ''}KT"

    def __iter__(self):
        yield from self.to_dict().values()


wind1 = Wind(10, 10, 15.0)
(
    wind1,
    wind1.to_dict(),
    wind1.to_dict().values(),
    wind1.to_dict().keys(),
    str(wind1),
    wind1.to_numpy(),
    wind1.to_pandas(),
)

(Wind(direction=10, speed=10, gust=15.0),
 {'direction': 10, 'speed': 10, 'gust': 15.0},
 dict_values([10, 10, 15.0]),
 dict_keys(['direction', 'speed', 'gust']),
 '01010G15KT',
 array([10., 10., 15.], dtype=float32),
 direction    10.0
 speed        10.0
 gust         15.0
 dtype: float32)

In [11]:
wind2 = Wind(350, 15, 25)
delta = wind1 - wind2
delta, abs(delta)

(Wind(direction=20.0, speed=-5.0, gust=-10.0),
 Wind(direction=20.0, speed=5.0, gust=10.0))

In [8]:
abs(delta), max([wind1, wind2]), min([wind1, wind2])

(Wind(direction=5.0, speed=5.0, gust=10.0),
 Wind(direction=15, speed=15, gust=25),
 Wind(direction=10, speed=10, gust=15.0))

In [9]:
print(
    f"the diff = {abs(delta)}\nthe max = {max([wind1,wind2])}\nthe min = {min([wind1,wind2])}"
)

the diff = 00505G10KT
the max = 01515G25KT
the min = 01010G15KT
